In [13]:
import datetime 
import numpy as np
import mpmath as mp
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
from sklearn.preprocessing import StandardScaler
%matplotlib qt

from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
import os 
from os import listdir
from os.path import isfile, join

def get_directory(folder):
    files= [a for a in listdir(folder) if isfile(join(folder,a))]
    
    return files
import os 
from os import listdir
from os.path import isfile, join

"get directory 2.0"

def get_directory_contents(folder):
    items = os.listdir(folder)
    files = [item for item in items if os.path.isfile(os.path.join(folder, item))]
    folders = [item for item in items if os.path.isdir(os.path.join(folder, item))]
    
    return {'files': files, 'folders': folders}

In [14]:
from datetime import datetime 

def search_the_day(fecha):
    # Fecha del primer día del año
    primer_dia_del_ano = datetime(fecha.year, 1, 1)

    # Calcula la diferencia entre las dos fechas
    diferencia = fecha - primer_dia_del_ano

    # Obtiene el número de días transcurridos
    dias_transcurridos = diferencia.days+1
    #dias_transcurridos = diferencia.days
    
    return dias_transcurridos
    

## ITERATION

In [15]:
main_folder = "C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019"


#print('main folder',main_folder)
list_of_files=get_directory_contents(main_folder)                 
folders=list_of_files['folders']

print(folders)

TOTALDATA=[]

for i in np.arange(0,len(folders)):
#for i in np.arange(0,4):
    main_folder2=main_folder+'/'+str(folders[i])
    print(main_folder2)
    list_of_files2=get_directory_contents(main_folder2)                   
    files=list_of_files2['files']
    
    newlist=[]
    for item in files:                                        #  Iteracion sobre cada elemento, elimina los archivos que no son de interes
        # Verificar si la cadena inicia con la letra "p"
        if item.startswith("p"):
            newlist.append(main_folder2+'/'+str(item))
    #print(newlist)
    
    #for j in np.arange(0,len(newlist)):
    
    'Iteracion for each hour of a day'
            
    list_of_data_per_day=[] #list with all the data per hour per day
    list_of_hours=[] 
    for j in np.arange(0,len(newlist)):
    #for j in np.arange(0,1):

        #-----------------------------------------------------------------------------------------------
        'Retain the date of the measurement'
        dato=pd.read_table(newlist[j]) 
        dato=dato.iloc[np.arange(2,3)][' PTS'][2]


        numeros = dato.split()
        numeros_enteros = [int(numero) for numero in numeros]
        numeros = numeros_enteros
        from datetime import datetime
        ano = 2000 + numeros[0]  
        mes = numeros[1]
        dia = numeros[2]
        hora = numeros[3]
        minuto = numeros[4]
        segundo = numeros[5]

        fecha_hora = datetime(ano, mes, dia, hora, minuto, segundo)
                    #print('fecha_hora',fecha_hora)
                    #print('fecha_hora',fecha_hora.hour)
        list_of_hours.append(fecha_hora)
        #-----------------------------------------------------------------------------------------------

        'Wind data'

        nombrearchivo = newlist[j] # Name of the file
        titles=['HT',      'SPD',      'DIR',    'MET_QC'  ,'RAD1',      'RAD2',      'RAD3',      'CNT1',      'CNT2',      'CNT3',      'SNR1',      'SNR2',      'SNR3', 'QC1', 'QC2', 'QC3'   ]
        data=pd.read_table(nombrearchivo, skiprows=10 ,header=None,names=titles, sep='\s+') #,parse_dates=['t_utc'])
        data=data.iloc[np.arange(1,len(data)),:] #delete the first row
        data=data.reset_index(drop=True)

        'Eliminate nans'
        data=data.astype(object)
        data = data.dropna()
        data=data.reset_index(drop=True)

        'Drop the rest of dataframes if there is more than 1'

        for k in np.arange(0,len(data)):
            if str(data['HT'][k])=='HT':
                data=data.iloc[np.arange(0,k),:] 
                break

        'Eliminar los 99999'
        for k in np.arange(0,len(data)):
            if float(data['SPD'][k])==999999 or float(data['HT'][k])>=1.5:
                data.drop(k, axis=0, inplace=True)

        data=data.reset_index(drop=True)


        'moving average'
        data2=data[['HT','SPD','DIR']]

        window_size = 2  # Tamaño de la ventana de media móvil

        # Aplicar filtro de media móvil
        data['SPDsmoothed'] = data2['SPD'].rolling(window=window_size, min_periods=1).mean()
        data['DIRsmoothed'] = data2['DIR'].rolling(window=window_size, min_periods=1).mean()

        list_of_data_per_day.append(data)

    TOTALDATA.append( [list_of_data_per_day, list_of_hours] )
    

['027', '028', '029', '030', '031', '032', '033', '035', '036', '037', '038', '039', '040', '042', '043', '044', '049', '050', '051', '052', '053', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177'

In [16]:
#print(TOTALDATA)
print(len(TOTALDATA[0][0]))
print(TOTALDATA[0][1][6])

7
2019-01-27 23:00:09


### PLOTS

In [17]:
plt.figure()
for k in np.arange(0,90): #for each day
#for k in np.arange(0,len(TOTALDATA)): #for each day    
    for l in np.arange(0,len(TOTALDATA[k][0])):

        x=TOTALDATA[k][0][l]['HT'].astype(float)
        x=x.round(decimals=1, out=None)
        windspeed=TOTALDATA[k][0][l]['SPD'].astype(float)
        windspeedmean=TOTALDATA[k][0][l]['SPDsmoothed'].astype(float)
        windangles=TOTALDATA[k][0][l]['DIR'].astype(float)
        windanglesmean=TOTALDATA[k][0][l]['DIRsmoothed'].astype(float)
        ysize=15

        #plt.title('Hour nº'+str(a+1))


        ax1=plt.subplot(211)
        plt.plot(x,windspeed,'.',c='black')
                            #plt.plot(x,windspeedmean,'-',c='red')
        ax1.tick_params(axis="y", labelsize=ysize, labelcolor='black')
        ax1.tick_params(axis="x", labelsize=ysize, labelcolor='black')  
        plt.ylabel('Wind speed (m/s)',fontsize=ysize,color='black')  
        plt.setp(ax1.get_xticklabels(), visible=False)

        ax2=plt.subplot(212,sharex=ax1)
        plt.plot(x,windangles,'.',c='purple')
                            #plt.plot(x,windanglesmean,'-',c='blue')
        ax2.tick_params(axis="y", labelsize=ysize, labelcolor='black')  
        ax2.tick_params(axis="x", labelsize=ysize, labelcolor='black') 
        plt.ylabel('Wind direction (deg)',fontsize=ysize,color='black')
        plt.xlabel('Height above ground (km)',fontsize=ysize,color='black') 



In [18]:
'promedio de cada altura'

'separar por dias y despues por hora del dia'


hora1=0
hora2=6

datahistograms=[]

plt.figure()
'primeros 3 meses 0 a 57'
'57 a 146'
'146 a 234'
'234 a len()'


for k in np.arange(0,57): #for each day    
#for k in np.arange(0,len(TOTALDATA)): #for each day    
    for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
        
        if TOTALDATA[k][1][l].hour>=hora1 and TOTALDATA[k][1][l].hour<=hora2: #if we are in the right time interval

            x=TOTALDATA[k][0][l]['HT'].astype(float)
            x=x.round(decimals=1, out=None)
            windspeed=TOTALDATA[k][0][l]['SPD'].astype(float)
            windspeedmean=TOTALDATA[k][0][l]['SPDsmoothed'].astype(float)
            windangles=TOTALDATA[k][0][l]['DIR'].astype(float)
            windanglesmean=TOTALDATA[k][0][l]['DIRsmoothed'].astype(float)
            ysize=15
            
            ax1=plt.subplot(211)
            plt.plot(x,windspeed,'.',c='black')
                                #plt.plot(x,windspeedmean,'-',c='red')
            ax1.tick_params(axis="y", labelsize=ysize, labelcolor='black')
            ax1.tick_params(axis="x", labelsize=ysize, labelcolor='black')  
            plt.ylabel('Wind speed (m/s)',fontsize=ysize,color='black')  
            plt.setp(ax1.get_xticklabels(), visible=False)

            ax2=plt.subplot(212,sharex=ax1)
            plt.plot(x,windangles,'.',c='purple')
                                #plt.plot(x,windanglesmean,'-',c='blue')
            ax2.tick_params(axis="y", labelsize=ysize, labelcolor='black')  
            ax2.tick_params(axis="x", labelsize=ysize, labelcolor='black') 
            plt.ylabel('Wind direction (deg)',fontsize=ysize,color='black')
            plt.xlabel('Height above ground (km)',fontsize=ysize,color='black') 
            
            
            for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                datahistograms.append( [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z]) ] )
            
            

In [19]:


km1=0.190
km2=0.210

def histogram(datahistograms,km1,km2):
    
    data=[]
    for i in np.arange(0,len(datahistograms)):
        if km1<=datahistograms[i][0]<=km2:
            data.append(datahistograms[i][1]) 
    plt.figure()   
    plt.hist(data)
    plt.title('Histogram ['+ str(km1) +','+str(km2) + '] km')
    plt.ylabel('Count')
    plt.xlabel('Wind speed (m/s)')
    
    return

def MeanAndStd(datahistograms,km1,km2):
    data=[]
    for i in np.arange(0,len(datahistograms)):
        if km1<=datahistograms[i][0]<=km2:
            data.append(datahistograms[i][1]) 
    mean=np.nanmean(data)
    std=np.nanstd(data)
    
    return  mean, std

histogram(datahistograms,0.180,0.220)  
histogram(datahistograms,0.280,0.320)  
histogram(datahistograms,0.380,0.420)
histogram(datahistograms,0.480,0.520)
histogram(datahistograms,0.580,0.620)
histogram(datahistograms,0.680,0.720)
histogram(datahistograms,0.780,0.820)
histogram(datahistograms,0.880,0.920)
histogram(datahistograms,0.980,1.020)
histogram(datahistograms,1.080,1.120)

histogram(datahistograms,1.180,1.220)
histogram(datahistograms,1.280,1.320)
histogram(datahistograms,1.380,1.420)

In [20]:
def ITERATION(datahistograms):
    a1=[[0.180,0.220], [0.280,0.320], [0.380,0.420],[0.480,0.520],[0.580,0.620],[0.680,0.720],[0.780,0.820],[0.880,0.920],[0.980,1.020],[1.080,1.120],[1.180,1.220],[1.280,1.320],[1.380,1.420]]
    
    meanlist=[]
    stdlist=[]
    for i in np.arange(0,len(a1)):
        mean,std= MeanAndStd(datahistograms,a1[i][0],a1[i][1])
        meanlist.append(mean)
        stdlist.append(std)
        
    return meanlist, stdlist, a1
    

In [21]:
means,deviations,kilometers= ITERATION(datahistograms)

kilometersplot=[]
for i in np.arange(0,len(kilometers)):
    kilometersplot.append( (kilometers[i][0]+kilometers[i][1])/2 )
    
    
plt.figure()
plt.errorbar(kilometersplot, means, yerr=deviations, linestyle='-', marker='o', capsize=5, label='Datos con error')
plt.xlabel('Altitude (km)')
plt.ylabel('Wind Speed Mean Values (m/s) ')
    

Text(0, 0.5, 'Wind Speed Mean Values (m/s) ')

In [22]:
print(datahistograms)

[[0.195, 2.1], [0.297, 2.1], [0.398, 1.8], [0.499, 2.5], [0.601, 4.5], [0.702, 6.0], [0.803, 5.7], [0.905, 5.4], [1.006, 5.8], [1.107, 5.7], [1.209, 5.4], [1.31, 4.3], [1.411, 3.4], [0.195, 1.9], [0.297, 2.5], [0.398, 3.0], [0.499, 4.2], [0.601, 4.7], [0.702, 4.9], [0.803, 5.2], [0.905, 5.8], [1.006, 6.8], [1.107, 6.6], [1.209, 5.6], [1.31, 4.8], [1.411, 4.1], [0.195, 1.8], [0.297, 2.6], [0.398, 3.7], [0.499, 3.8], [0.601, 2.7], [0.702, 3.1], [0.803, 4.2], [0.905, 5.1], [1.006, 6.2], [1.107, 6.3], [1.209, 5.2], [1.31, 4.8], [1.411, 4.5], [0.195, 1.9], [0.297, 1.9], [0.398, 2.0], [0.499, 1.3], [0.601, 2.1], [0.702, 3.2], [0.803, 4.0], [0.905, 5.6], [1.006, 7.9], [1.107, 8.3], [1.209, 6.2], [1.31, 5.0], [1.411, 5.3], [0.195, 1.5], [0.297, 1.7], [0.398, 1.4], [0.499, 1.2], [0.601, 1.5], [0.702, 2.3], [0.803, 3.1], [0.905, 4.5], [1.006, 5.9], [1.107, 7.0], [1.209, 7.0], [1.31, 6.1], [1.411, 5.9], [0.195, 0.5], [0.297, 0.9], [0.398, 1.7], [0.499, 2.9], [0.601, 3.7], [0.702, 4.0], [0.803, 3.

# GLOBAL FUNCTION

In [23]:
def GlobalFunction(main_folder,hora1,hora2):
    
    list_of_files=get_directory_contents(main_folder)                 
    folders=list_of_files['folders']

    print(folders)

    TOTALDATA=[]

    for i in np.arange(0,len(folders)):
    #for i in np.arange(0,4):
        main_folder2=main_folder+'/'+str(folders[i])
        print(main_folder2)
        list_of_files2=get_directory_contents(main_folder2)                   
        files=list_of_files2['files']

        newlist=[]
        for item in files:                                        #  Iteracion sobre cada elemento, elimina los archivos que no son de interes
            # Verificar si la cadena inicia con la letra "p"
            if item.startswith("p"):
                newlist.append(main_folder2+'/'+str(item))
        #print(newlist)

        #for j in np.arange(0,len(newlist)):

        'Iteracion for each hour of a day'

        list_of_data_per_day=[] #list with all the data per hour per day
        list_of_first_derivative_per_day=[]
        list_of_second_derivative_per_day=[]
        list_of_hours=[] 
        for j in np.arange(0,len(newlist)):
        #for j in np.arange(0,1):

            #-----------------------------------------------------------------------------------------------
            'Retain the date of the measurement'
            dato=pd.read_table(newlist[j]) 
            dato=dato.iloc[np.arange(2,3)][' PTS'][2]


            numeros = dato.split()
            numeros_enteros = [int(numero) for numero in numeros]
            numeros = numeros_enteros
            from datetime import datetime
            ano = 2000 + numeros[0]  
            mes = numeros[1]
            dia = numeros[2]
            hora = numeros[3]
            minuto = numeros[4]
            segundo = numeros[5]

            fecha_hora = datetime(ano, mes, dia, hora, minuto, segundo)
                        #print('fecha_hora',fecha_hora)
                        #print('fecha_hora',fecha_hora.hour)
            list_of_hours.append(fecha_hora)
            #-----------------------------------------------------------------------------------------------

            'Wind data'

            nombrearchivo = newlist[j] # Name of the file
            titles=['HT',      'SPD',      'DIR',    'MET_QC'  ,'RAD1',      'RAD2',      'RAD3',      'CNT1',      'CNT2',      'CNT3',      'SNR1',      'SNR2',      'SNR3', 'QC1', 'QC2', 'QC3'   ]
            data=pd.read_table(nombrearchivo, skiprows=10 ,header=None,names=titles, sep='\s+') #,parse_dates=['t_utc'])
            data=data.iloc[np.arange(1,len(data)),:] #delete the first row
            data=data.reset_index(drop=True)

            'Eliminate nans'
            data=data.astype(object)
            data = data.dropna()
            data=data.reset_index(drop=True)

            'Drop the rest of dataframes if there is more than 1'

            for k in np.arange(0,len(data)):
                if str(data['HT'][k])=='HT':
                    data=data.iloc[np.arange(0,k),:] 
                    break

            'Eliminar los 99999'
            for k in np.arange(0,len(data)):
                if float(data['SPD'][k])==999999 or float(data['HT'][k])>=1.5:
                    data.drop(k, axis=0, inplace=True)

            data=data.reset_index(drop=True)


            'moving average'
            data2=data[['HT','SPD','DIR']]

            window_size = 2  # Tamaño de la ventana de media móvil

            # Aplicar filtro de media móvil
            data['SPDsmoothed'] = data2['SPD'].rolling(window=window_size, min_periods=1).mean()
            data['DIRsmoothed'] = data2['DIR'].rolling(window=window_size, min_periods=1).mean()

            
            # let's see what happens with the first and second derivative CENTERED DERIVATIVE 
            
            first_ht=[]   
            first_derivative=[]
            second_ht=[]  
            second_derivative=[]
            
            data['HT']=data['HT'].astype(float)
            data['SPD']=data['SPD'].astype(float)
            
            for k in np.arange(0,len(data['HT'])-1):
                first_ht.append(data['HT'][k])
                derivative= ( (data['SPD'][k+1]-data['SPD'][k])/(data['HT'][k+1]-data['HT'][k]) )
                first_derivative.append(derivative)
            
            for k in np.arange(1,len(data['HT'])-1):
                second_ht.append(data['HT'][k])
                
                derivative= ( (data['SPD'][k+1]-2*data['SPD'][k]+data['SPD'][k-1]) / ((data['HT'][k+1]-data['HT'][k])**2) )
                second_derivative.append(derivative)
            
            
            d = {'HT_first': first_ht, 'SPD_first': first_derivative}
            FIRST_DERIVATIVE_TABLE = pd.DataFrame(data=d)
            
            dd = {'HT_second': second_ht, 'SPD_second': second_derivative}
            SECOND_DERIVATIVE_TABLE = pd.DataFrame(data=dd)
            
            #print(FIRST_DERIVATIVE_TABLE)
            #print(SECOND_DERIVATIVE_TABLE)
            
            
            list_of_data_per_day.append(data)
            list_of_first_derivative_per_day.append(FIRST_DERIVATIVE_TABLE)
            list_of_second_derivative_per_day.append(SECOND_DERIVATIVE_TABLE)

        TOTALDATA.append( [list_of_data_per_day, list_of_hours, list_of_first_derivative_per_day,list_of_second_derivative_per_day ] )
 
    #-------------------------------------------------------------------------------------------------
    datahistograms=[] 
    datahistograms1st=[] 
    datahistograms2nd=[] 
    for k in np.arange(0,len(TOTALDATA)): #for each day    
        for l in np.arange(0,len(TOTALDATA[k][0])): #for each hour
            if TOTALDATA[k][1][l].hour>=hora1 and TOTALDATA[k][1][l].hour<=hora2: #if we are in the right time interval
                for z in np.arange(0,len(TOTALDATA[k][0][l]['HT'])):
                    datahistograms.append( [float(TOTALDATA[k][0][l]['HT'][z]), float(TOTALDATA[k][0][l]['SPD'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][2][l]['HT_first'])):
                    datahistograms1st.append( [float(TOTALDATA[k][2][l]['HT_first'][z]), float(TOTALDATA[k][2][l]['SPD_first'][z]) ] )
                
                for z in np.arange(0,len(TOTALDATA[k][3][l]['HT_second'])):
                    datahistograms2nd.append( [float(TOTALDATA[k][3][l]['HT_second'][z]), float(TOTALDATA[k][3][l]['SPD_second'][z]) ] )    
    #-------------------------------------------------------------------------------------------------
    def MeanAndStd(datahistograms,km1,km2):
        data=[]
        for i in np.arange(0,len(datahistograms)):
            if km1<=datahistograms[i][0]<=km2:
                data.append(datahistograms[i][1]) 
        mean=np.nanmean(data)
        std=np.nanstd(data)

        return  mean, std
    
    def ITERATION(datahistograms,datahistograms1st,datahistograms2nd):
        a1=[[0.180,0.220], [0.280,0.320], [0.380,0.420],[0.480,0.520],[0.580,0.620],[0.680,0.720],[0.780,0.820],[0.880,0.920],[0.980,1.020],[1.080,1.120],[1.180,1.220],[1.280,1.320],[1.380,1.420]]

        meanlist=[]
        stdlist=[]
        
        meanlist1st=[]
        stdlist1st=[]
        
        meanlist2nd=[]
        stdlist2nd=[]
        
        for i in np.arange(0,len(a1)):
            mean,std= MeanAndStd(datahistograms,a1[i][0],a1[i][1])
            meanlist.append(mean)
            stdlist.append(std)
            
        for i in np.arange(0,len(a1)-1):
            mean,std= MeanAndStd(datahistograms1st,a1[i][0],a1[i][1])
            meanlist1st.append(mean)
            stdlist1st.append(std)
            
        for i in np.arange(1,len(a1)-1):
            mean,std= MeanAndStd(datahistograms2nd,a1[i][0],a1[i][1])
            meanlist2nd.append(mean)
            stdlist2nd.append(std)    
            

        return meanlist, stdlist, a1,  meanlist1st,stdlist1st,  meanlist2nd,stdlist2nd
    #-------------------------------------------------------------------------------------------------
    means,deviations,kilometers , means1st,deviations1st,  means2nd,deviations2nd = ITERATION(datahistograms, datahistograms1st, datahistograms2nd)
    
    kilometersplot=[]
    kilometersplot2=[]
    kilometersplot3=[]
    for i in np.arange(0,len(kilometers)):
        kilometersplot.append( (kilometers[i][0]+kilometers[i][1])/2 )
    for i in np.arange(0,len(kilometers)-1):
        kilometersplot2.append( (kilometers[i][0]+kilometers[i][1])/2 )
    for i in np.arange(1,len(kilometers)-1):
        kilometersplot3.append( (kilometers[i][0]+kilometers[i][1])/2 )    


    plt.figure()
    plt.errorbar(kilometersplot, means, yerr=deviations, linestyle='-', marker='o', capsize=5, label='Datos con error')
    plt.xlabel('Altitude (km)')
    plt.ylabel('Wind Speed Mean Values (m/s) ')
    
    plt.figure()
    plt.errorbar(kilometersplot2, means1st, yerr=deviations1st, linestyle='-', marker='o', capsize=5, label='Datos con error')
    plt.xlabel('Altitude (km)')
    plt.ylabel('Wind Speed Mean Values (m/s km) ')
    plt.title('First derivative')
    
    plt.figure()
    plt.errorbar(kilometersplot3, means2nd, yerr=deviations2nd, linestyle='-', marker='o', capsize=5, label='Datos con error')
    plt.xlabel('Altitude (km)')
    plt.ylabel('Wind Speed Mean Values (m/s km2) ')
    plt.title('Second derivative')
    
    return

In [24]:
GlobalFunction('C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5',18,19)

['030', '031', '032', '033', '035', '036', '037', '038', '039', '040', '042', '043', '044', '049', '050', '051', '052', '053', '055', '056', '057', '058', '059', '060']
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/030
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/031
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/032
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/033
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/035
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/036
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/037
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/038
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/039
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/040
C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019subsets/5/042
C:/Use